In [7]:
# novelist.ipynb
# 프로젝트: 4. 작사가 인공지능 만들기

''' 
프로젝트: 4. 작사가 인공지능 만들기

순환신경망(RNN)과 tensorflow LSTM을 이용하여 한줄의 가사를 작사합니다.

'''

import os, re 
import numpy as np
import tensorflow as tf
import glob
from sklearn.model_selection import train_test_split

MAX_TOKEN_SIZE = 15 #15 단어 이상의 문장은 제외

    
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence


def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    
#   단어장의 크기는 12,000    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post') 

#     print(tensor,tokenizer)
    return tensor, tokenizer    
    
    
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated    
    

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out    
    


# Step 1. 데이터 다운로드

# `실습(1) 데이터 다듬기`에서 `Cloud shell`에 심볼릭 링크로 `
# ~/aiffel/lyricist/data`를 생성하셨다면, 
# `~/aiffel/lyricist/data/lyrics`에 데이터가 있습니다.    
    
    

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


Step 2. 데이터 읽어오기

In [ ]:

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)    

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)


print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

Step 3. 데이터 정제

In [8]:


corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    
    # 토큰 사이즈가 MAX_TOKEN_SIZE = 15가 넘으면 제외
    if preprocessed_sentence.count(' ') < MAX_TOKEN_SIZE:
        corpus.append(preprocessed_sentence)
        

tensor, tokenizer = tokenize(corpus)    
print(tensor[:3, :10]) # 생성된 텐서 데이터를 3번째 행, 10번째 열까지만 출력    



# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]         
    

[[   2   50    4   95  303   62   53    9  946 6263]
 [   2   15 2967  871    5    8   11 5739    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]


Step 4. 평가 데이터셋 분리. 총 데이터의 20% 를 평가 데이터셋으로 사용

In [10]:

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True, random_state=34) 

print("Source Train:", enc_train.shape)
#Source Train: (124960, 14)
print("Target Train:", dec_train.shape)
#Target Train: (124960, 14)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [12]:

BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset
#<BatchDataset shapes: ((256, 20), (256, 20)), types: (tf.int32, tf.int32)>




embedding_size = 256 # 워드벡터의 차원수, 즉 단어가 추상적으로 표현되는 크기
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)




# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)



model.summary()




optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
487/487 [==============================] - 82s 164ms/step - loss: 3.4900
Epoch 2/10
487/487 [==============================] - 82s 168ms/step - loss: 3.0266
Epoch 3/10
487/487 [==============================] - 82s 168ms/step - loss: 2.8561
Epoch 4/10
487/487 [==============================] - 82s 168ms/step - loss: 2.7254
Epoch 5/10
487/487 [==============================] - 82s 168ms/step - loss: 2.6173
Epoch 6/10
487/487 [==============================] - 82s 168ms/step - loss: 2.5214
Epoch 7/10
487/487 [==============================] - 82s 168ms/step - loss: 2.4330
Epoch 8/10
487/487 [==============================] - 82s 169ms/step - loss: 2.3492
Epoch 9/10
487/487 [==============================] - 82s 168ms/step - loss: 2.2701
Epoch 10/10
487/487 [==============================] - 82s 169ms/step - loss: 2.1950


In [14]:
#잘만들어졌는가 확인 
generate_text(model, tokenizer, init_sentence="<start> i love")

'<start> i love you , i m a liability <end> '

In [15]:
generate_text(model, tokenizer, init_sentence="<start> you are")

'<start> you are the only one who knows <end> '

In [20]:
generate_text(model, tokenizer, init_sentence="<start> love is")

'<start> love is a beautiful thing <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> time")

'<start> time to get you up <end> '

In [25]:
generate_text(model, tokenizer, init_sentence="<start> black")

'<start> black and white , puerto rican , chinese <end> '


1 테스트셋을 이용해서 정확도를 구한다면 단어 뒤에 따라오는 단어를 비교할지 어쩔지 고민하다 포기했다.

2 시간이 좀 걸리는 과정이라 여러가지 시도를 해보지는 못했다.

3 우선 loss: 2.1950 < 2  나오는 것 보고 더이상 진행은 하지 않았다.

 